In [20]:
import pandas as pd
import numpy as np
import os

# Load the data
data1 = pd.read_excel('/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/raw/2024_11_12_clean_data_transformed_incl_MOA_and_compound.xlsx')
data1[['Part_1', 'Part_2', 'Part_3']] = data1['Treatment'].str.split('.', expand=True)

# Create two new folders for the output files
output_folder_exp = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/intermediate/transposed_dataframes'
output_folder_filled = '/Users/yahyafarooqi/Documents/Code/AntiGEM/FluxGraph/Data/intermediate/transposed_dataframes_filled'

os.makedirs(output_folder_exp, exist_ok=True)
os.makedirs(output_folder_filled, exist_ok=True)

# Define rows to exclude from the dataframe after transposing
rows_to_exclude = ['Treatment', 'a priori MOA categorization', 'compound', 'Part_1', 'Part_2', 'Part_3']

# Iterate over each unique value in the 'Part_1' column
for part_1_value in data1['Part_1'].unique():
    # Filter rows where 'Part_1' column matches the current value
    filtered_df = data1[data1['Part_1'] == part_1_value]
    filtered_df.columns = filtered_df.columns.str.replace('BSU_', 'BSU', regex=False) 
    filtered_df = filtered_df.loc[:, filtered_df.columns.str.contains('BSU')]


        # Transform numeric values using exponential function and exclude specified rows
    filtered_df_exp = filtered_df.applymap(lambda x: np.exp(x) if np.issubdtype(type(x), np.number) and pd.notnull(x) else x)
    transposed_df_exp = filtered_df_exp.transpose()
    transposed_df_exp = transposed_df_exp[~transposed_df_exp.index.isin(rows_to_exclude)]
    file_name_exp = f"{output_folder_exp}/{part_1_value}_transposed.tsv"
    transposed_df_exp.to_csv(file_name_exp, header=False, sep='\t')

    # Transpose the filtered DataFrame and fill missing values
    filtered_df_exp2 = filtered_df.applymap(lambda x: np.exp(x) if np.issubdtype(type(x), np.number) and pd.notnull(x) else x)
    transposed_df_filled = filtered_df_exp2.transpose()
    transposed_df_filled = transposed_df_filled.apply(lambda row: row.fillna(row.mean(skipna=True)) if row.isnull().any() else row, axis=1)
    file_name_filled = f"{output_folder_filled}/{part_1_value}_transposed_filled.tsv"
    # if row still has missing values, remove it
    transposed_df_filled = transposed_df_filled.dropna(axis=0)
    transposed_df_filled.to_csv(file_name_filled, header=False, sep='\t')